In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, InputLayer
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K
from keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import nasnet, resnet50, densenet
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

%matplotlib inline

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine') 

In [3]:
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

In [6]:
base_model = densenet.DenseNet121(weights='imagenet', input_shape=(32,32,3), include_top=False)
# base_model.summary()

In [7]:
base_model.input, base_model.output

(<tf.Tensor 'input_3:0' shape=(None, 32, 32, 3) dtype=float32>,
 <tf.Tensor 'relu_2/Relu:0' shape=(None, 1, 1, 1024) dtype=float32>)

In [9]:
tensor = Flatten()(base_model.output)
tensor

<tf.Tensor 'flatten_1/Reshape:0' shape=(None, 1024) dtype=float32>

In [10]:
tensor = Dense(100, activation='softmax')(tensor)
tensor

<tf.Tensor 'dense/Softmax:0' shape=(None, 100) dtype=float32>

In [13]:
model = Model(inputs=base_model.input, outputs=tensor)
# model.summary()

In [17]:
model.layers[:5]

In [20]:
[l.trainable for l in model.layers][:5]

[True, True, True, True, True]

In [22]:
for i, layer in enumerate(model.layers):
    if i < 427:
        layer.trainable = False
    else:
        layer.trainable = True
    # print(i, layer, layer.trainable)

In [24]:
opt = optimizers.Nadam()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['categorical_accuracy'])
# model.summary()

In [25]:
history = model.fit(x_train, y_train, epochs=3, batch_size=32, verbose=2, validation_split=0.3, shuffle=True)

Train on 35000 samples, validate on 15000 samples
Epoch 1/3
35000/35000 - 281s - loss: 3.7418 - categorical_accuracy: 0.1735 - val_loss: 3.7231 - val_categorical_accuracy: 0.1955
Epoch 2/3
35000/35000 - 286s - loss: 3.0349 - categorical_accuracy: 0.2804 - val_loss: 3.8645 - val_categorical_accuracy: 0.1955
Epoch 3/3
35000/35000 - 273s - loss: 2.8640 - categorical_accuracy: 0.3085 - val_loss: 3.9377 - val_categorical_accuracy: 0.2039
